## Proyecto API

In [1]:
import requests
import json
import pymongo

In [15]:
for i in range(41):
    url = "https://api.potterdb.com/v1/characters?page[number]="+str(i+1)

    # Make the API request
    response = requests.get(url)
    # print(response.text)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the response JSON data
        data = response.json()

        # Save the data to a file
        with open("hp_data.json", "w") as file:
            json.dump(data, file)
            print("Data downloaded and saved successfully.")
    else:
        print("Failed to retrieve data from the API. Status code:", response.status_code)
        print("Error message:", response.text)


    # Set up a MongoDB client and database
    client = pymongo.MongoClient('mongodb://localhost:27017/')
    db = client['potter']

    # Insert a document into a collection
    my_collection = db['characters']
    my_data = [data]
    insert_result = my_collection.insert_many(my_data)
# # Print the data
# print(data)

{"data":[{"id":"a3701158-39ed-4687-8b9a-a799cfca5811","type":"character","attributes":{"slug":"1992-gryffindor-vs-slytherin-quidditch-match-spectators","name":"1992 Gryffindor vs Slytherin Quidditch match spectators","born":null,"died":null,"gender":null,"species":null,"height":null,"weight":null,"hair_color":null,"eye_color":null,"skin_color":null,"blood_status":null,"marital_status":null,"nationality":null,"animagus":null,"boggart":null,"house":null,"patronus":null,"alias_names":null,"family_members":null,"jobs":null,"romances":null,"titles":null,"wands":null,"image":null,"wiki":"https://harrypotter.fandom.com/wiki/1992_Gryffindor_vs_Slytherin_Quidditch_match_spectators"}},{"id":"e06d8f25-fae5-4970-884e-06ae2f53d707","type":"character","attributes":{"slug":"1996-gryffindor-quidditch-keeper-trials-spectators","name":"1996 Gryffindor Quidditch Keeper trials spectators","born":null,"died":null,"gender":null,"species":null,"height":null,"weight":null,"hair_color":null,"eye_color":null,"s

## Importar a mongo

In [17]:
import random
seed = 1234

pipeline = [
    {"$unwind": "$data"},
    {"$set": {"_id": "$data.id"+str(random.randint)}},
    {"$out": "char"}
]
db.characters.aggregate(pipeline)

In [18]:
pipeline2 = [
    {
        '$replaceWith': {
            '$mergeObjects': ['$data', '$$ROOT']
        }
    },
    {
        '$unset': 'data'
    }
]

db.char.update_many({}, pipeline2)

In [19]:
db.char.update_many({}, {'$unset': {'data.id': 1}})

## De mongo a un .json

In [4]:
from bson import ObjectId

# Custom JSON encoder class to handle ObjectId serialization
class JSONEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, ObjectId):
            return str(o)
        return super().default(o)

# Connect to your MongoDB database
client = pymongo.MongoClient('mongodb://localhost:27017/')
db = client['potter']
collection = db['char']

# Fetch the documents from the collection
documents = collection.find()

# Convert the documents to a list of dictionaries
documents_list = [doc for doc in documents]

# Save the documents to a JSON file
with open('exported_data.json', 'w') as file:
    json.dump(documents_list, file, indent=4, cls=JSONEncoder)



In [2]:
import pandas as pd

# Opening JSON file
f = open('exported_data.json')

# returns JSON object as
# a dictionary
data = json.load(f)

# Closing file
f.close()
# data

In [3]:
df = pd.json_normalize(data)
df

,_id,id,type,attributes.slug,attributes.name,attributes.born,attributes.died,attributes.gender,attributes.species,attributes.height,...,meta.copyright,meta.generated_at,links.self,links.current,links.next,links.last,meta.pagination.first,meta.pagination.prev,links.first,links.prev
0,646abd71f555c8e319b2ff9c,a3701158-39ed-4687-8b9a-a799cfca5811,character,1992-gryffindor-vs-slytherin-quidditch-match-s...,1992 Gryffindor vs Slytherin Quidditch match s...,None,None,None,None,None,...,© 2023 Potter DB API,2023-05-22T00:49:37.351Z,https://api.potterdb.com/v1/characters?page%5B...,https://api.potterdb.com/v1/characters?page[nu...,https://api.potterdb.com/v1/characters?page[nu...,https://api.potterdb.com/v1/characters?page[nu...,NaN,NaN,NaN,NaN
1,646abd71f555c8e319b2ff9d,e06d8f25-fae5-4970-884e-06ae2f53d707,character,1996-gryffindor-quidditch-keeper-trials-specta...,1996 Gryffindor Quidditch Keeper trials specta...,None,None,None,None,None,...,© 2023 Potter DB API,2023-05-22T00:49:37.351Z,https://api.potterdb.com/v1/characters?page%5B...,https://api.potterdb.com/v1/characters?page[nu...,https://api.potterdb.com/v1/characters?page[nu...,https://api.potterdb.com/v1/characters?page[nu...,NaN,NaN,NaN,NaN
2,646abd71f555c8e319b2ff9e,5e97c6a0-e000-4fb9-8d66-37cd6942d103,character,unidentified-19th-century-flying-instructor,19th-century Flying Instructor,Before 1890,None,Female,Human,None,...,© 2023 Potter DB API,2023-05-22T00:49:37.351Z,https://api.potterdb.com/v1/characters?page%5B...,https://api.potterdb.com/v1/characters?page[nu...,https://api.potterdb.com/v1/characters?page[nu...,https://api.potterdb.com/v1/characters?page[nu...,NaN,NaN,NaN,NaN
3,646abd71f555c8e319b2ff9f,fb7aad1b-50d7-4362-a315-d7432ae35372,character,unidentified-19th-century-herbology-professor,19th-century Herbology professor,None,None,Female,Human,None,...,© 2023 Potter DB API,2023-05-22T00:49:37.351Z,https://api.potterdb.com/v1/characters?page%5B...,https://api.potterdb.com/v1/characters?page[nu...,https://api.potterdb.com/v1/characters?page[nu...,https://api.potterdb.com/v1/characters?page[nu...,NaN,NaN,NaN,NaN
4,646abd71f555c8e319b2ffa0,41812dca-8341-4369-b286-5e27f35274b4,character,unidentified-19th-century-potions-professor,19th-century Potions professor,None,None,Male,Human,None,...,© 2023 Potter DB API,2023-05-22T00:49:37.351Z,https://api.potterdb.com/v1/characters?page%5B...,https://api.potterdb.com/v1/characters?page[nu...,https://api.potterdb.com/v1/characters?page[nu...,https://api.potterdb.com/v1/characters?page[nu...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4061,646abd71f555c8e319b30f79,936e7daf-ab6c-4d09-903e-0ba8091c336a,character,zoo-director,Zoo director,None,None,Male,Human,None,...,© 2023 Potter DB API,2023-05-22T00:50:20.595Z,https://api.potterdb.com/v1/characters?page%5B...,https://api.potterdb.com/v1/characters?page[nu...,NaN,NaN,1.0,40.0,https://api.potterdb.com/v1/characters?page[nu...,https://api.potterdb.com/v1/characters?page[nu...
4062,646abd71f555c8e319b30f7a,65853c0b-6990-4908-93bb-a4f3139ea181,character,zubeida-khan,Zubeida Khan,"1 September 1975 - 31 August 1982,",None,Female,Human,None,...,© 2023 Potter DB API,2023-05-22T00:50:20.595Z,https://api.potterdb.com/v1/characters?page%5B...,https://api.potterdb.com/v1/characters?page[nu...,NaN,NaN,1.0,40.0,https://api.potterdb.com/v1/characters?page[nu...,https://api.potterdb.com/v1/characters?page[nu...
4063,646abd71f555c8e319b30f7b,41e4f443-adf6-4935-a9f5-481e2657cf30,character,zygmunt-budge,Zygmunt Budge,"In one of the following years: 1490, 1491, 149...",None,Male,Human,None,...,© 2023 Potter DB API,2023-05-22T00:50:20.595Z,https://api.potterdb.com/v1/characters?page%5B...,https://api.potterdb.com/v1/characters?page[nu...,NaN,NaN,1.0,40.0,https://api.potterdb.com/v1/characters?page[nu...,https://api.potterdb.com/v1/characters?page[nu...
4064,646abd71f555c8e319b30f7c,8080ef61-4c6c-46c1-9cf0-629d16a23110,character,zygmunt-budge-

## ETL

In [5]:
df.columns

Index(['_id', 'id', 'type', 'attributes.slug', 'attributes.name',
       'attributes.born', 'attributes.died', 'attributes.gender',
       'attributes.species', 'attributes.height', 'attributes.weight',
       'attributes.hair_color', 'attributes.eye_color',
       'attributes.skin_color', 'attributes.blood_status',
       'attributes.marital_status', 'attributes.nationality',
       'attributes.animagus', 'attributes.boggart', 'attributes.house',
       'attributes.patronus', 'attributes.alias_names',
       'attributes.family_members', 'attributes.jobs', 'attributes.romances',
       'attributes.titles', 'attributes.wands', 'attributes.image',
       'attributes.wiki', 'meta.pagination.current', 'meta.pagination.next',
       'meta.pagination.last', 'meta.pagination.records', 'meta.copyright',
       'meta.generated_at', 'links.self', 'links.current', 'links.next',
       'links.last', 'meta.pagination.first', 'meta.pagination.prev',
       'links.first', 'links.prev'],
      dtype='

In [4]:
new_df = df.drop(['_id','meta.pagination.current', 'meta.pagination.next',
       'meta.pagination.last', 'meta.pagination.records', 'meta.copyright',
       'meta.generated_at', 'links.self', 'links.current', 'links.next',
       'links.last', 'meta.pagination.first', 'meta.pagination.prev',
       'links.first', 'links.prev','attributes.image',
       'attributes.wiki','attributes.height', 'attributes.weight'],axis = 1)

new_df

,id,type,attributes.slug,attributes.name,attributes.born,attributes.died,attributes.gender,attributes.species,attributes.hair_color,attributes.eye_color,...,attributes.animagus,attributes.boggart,attributes.house,attributes.patronus,attributes.alias_names,attributes.family_members,attributes.jobs,attributes.romances,attributes.titles,attributes.wands
0,a3701158-39ed-4687-8b9a-a799cfca5811,character,1992-gryffindor-vs-slytherin-quidditch-match-s...,1992 Gryffindor vs Slytherin Quidditch match s...,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,e06d8f25-fae5-4970-884e-06ae2f53d707,character,1996-gryffindor-quidditch-keeper-trials-specta...,1996 Gryffindor Quidditch Keeper trials specta...,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,5e97c6a0-e000-4fb9-8d66-37cd6942d103,character,unidentified-19th-century-flying-instructor,19th-century Flying Instructor,Before 1890,None,Female,Human,Black,None,...,None,None,None,None,None,None,[Flying Instructor at Hogwarts School of Witch...,None,None,None
3,fb7aad1b-50d7-4362-a315-d7432ae35372,character,unidentified-19th-century-herbology-professor,19th-century Herbology professor,None,None,Female,Human,Red,None,...,None,None,None,None,None,None,[Professor of Herbology at Hogwarts School of ...,None,None,"[Unknown length, wood and core]"
4,41812dca-8341-4369-b286-5e27f35274b4,character,unidentified-19th-century-potions-professor,19th-century Potions professor,None,None,Male,Human,Brown,None,...,None,None,None,None,None,None,[Potions Master at Hogwarts School of Witchcra...,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4061,936e7daf-ab6c-4d09-903e-0ba8091c336a,character,zoo-director,Zoo director,None,None,Male,Human,None,None,...,None,None,None,None,None,None,[Zoo director],None,[Director],None
4062,65853c0b-6990-4908-93bb-a4f3139ea181,character,zubeida-khan,Zubeida Khan,"1 September 1975 - 31 August 1982,",None,Female,Human,None,None,...,None,None,Slytherin,None,None,None,None,None,None,None
4063,41e4f443-adf6-4935-a9f5-481e2657cf30,character,zygmunt-budge,Zygmunt Budge,"In one of the following years: 1490, 1491, 149...",None,Male,Human,Dark,None,...,None,None,None,None,"['Fungus-face' (by his mother), The Hermit of ...","[Mrs Budge (mother), Great-grandmother]","[Author, Potioneer]",None,None,None
4064,8080ef61-4c6c-46c1-9cf0-629d16a23110,character,zygmunt-budge-s-great-grandmother,Zygmunt Budge's great-grandmother,None,None,Female,Human,Grey,None,...,None,None,None,None,None,[Mrs Budge (granddaughter or granddaughter-in-...,None,None,None,None


In [5]:
# Cambiamos el nombre de las columnas, eliminando 'attribute.'

columnas = new_df.columns.tolist()
new_columnas = [name.split(".")[1] if len(name.split(".")) > 1 else name for name in columnas]
new_df.columns = new_columnas

In [6]:
suma = new_df.isna().sum()
print(suma)

id                   0
type                 0
slug                 0
name                 0
born              1960
died              3390
gender             410
species            101
hair_color        2963
eye_color         3385
skin_color        2929
blood_status      2938
marital_status    3673
nationality       1866
animagus          4043
boggart           3967
house             3147
patronus          3878
alias_names       3866
family_members    2699
jobs              2437
romances          3899
titles            3283
wands             3795
dtype: int64


In [7]:
# Es una serie: key - nombre de la columna, value - tipo de dato
column_types = new_df.dtypes
column_types

id                object
type              object
slug              object
name              object
born              object
died              object
gender            object
species           object
hair_color        object
eye_color         object
skin_color        object
blood_status      object
marital_status    object
nationality       object
animagus          object
boggart           object
house             object
patronus          object
alias_names       object
family_members    object
jobs              object
romances          object
titles            object
wands             object
dtype: object

In [8]:
char = new_df.convert_dtypes()
print("Nuevos tipos:")
print(char.dtypes)

Nuevos tipos:
id                string
type              string
slug              string
name              string
born              string
died              string
gender            string
species           string
hair_color        string
eye_color         string
skin_color        string
blood_status      string
marital_status    string
nationality       string
animagus          string
boggart           string
house             string
patronus          string
alias_names       object
family_members    object
jobs              object
romances          object
titles            object
wands             object
dtype: object


In [9]:
import random

# new_df['house'].unique()

lista = ['Ravenclaw', 'Gryffindor', 'Slytherin', 'Hufflepuff', 'Unknown',
       'Thunderbird', 'Did not attend Hogwarts', 'Horned Serpent',
       'Hogwarts School of Witchcraft and Wizardry', 'Pukwudgie',
       'Wampus']
# ** ARREGLAR 
char['house'].fillna(random.choice(lista), inplace= True)
char

,id,type,slug,name,born,died,gender,species,hair_color,eye_color,...,animagus,boggart,house,patronus,alias_names,family_members,jobs,romances,titles,wands
0,a3701158-39ed-4687-8b9a-a799cfca5811,character,1992-gryffindor-vs-slytherin-quidditch-match-s...,1992 Gryffindor vs Slytherin Quidditch match s...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,Ravenclaw,<NA>,None,None,None,None,None,None
1,e06d8f25-fae5-4970-884e-06ae2f53d707,character,1996-gryffindor-quidditch-keeper-trials-specta...,1996 Gryffindor Quidditch Keeper trials specta...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,Ravenclaw,<NA>,None,None,None,None,None,None
2,5e97c6a0-e000-4fb9-8d66-37cd6942d103,character,unidentified-19th-century-flying-instructor,19th-century Flying Instructor,Before 1890,<NA>,Female,Human,Black,<NA>,...,<NA>,<NA>,Ravenclaw,<NA>,None,None,[Flying Instructor at Hogwarts School of Witch...,None,None,None
3,fb7aad1b-50d7-4362-a315-d7432ae35372,character,unidentified-19th-century-herbology-professor,19th-century Herbology professor,<NA>,<NA>,Female,Human,Red,<NA>,...,<NA>,<NA>,Ravenclaw,<NA>,None,None,[Professor of Herbology at Hogwarts School of ...,None,None,"[Unknown length, wood and core]"
4,41812dca-8341-4369-b286-5e27f35274b4,character,unidentified-19th-century-potions-professor,19th-century Potions professor,<NA>,<NA>,Male,Human,Brown,<NA>,...,<NA>,<NA>,Ravenclaw,<NA>,None,None,[Potions Master at Hogwarts School of Witchcra...,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4061,936e7daf-ab6c-4d09-903e-0ba8091c336a,character,zoo-director,Zoo director,<NA>,<NA>,Male,Human,<NA>,<NA>,...,<NA>,<NA>,Ravenclaw,<NA>,None,None,[Zoo director],None,[Director],None
4062,65853c0b-6990-4908-93bb-a4f3139ea181,character,zubeida-khan,Zubeida Khan,"1 September 1975 - 31 August 1982,",<NA>,Female,Human,<NA>,<NA>,...,<NA>,<NA>,Slytherin,<NA>,None,None,None,None,None,None
4063,41e4f443-adf6-4935-a9f5-481e2657cf30,character,zygmunt-budge,Zygmunt Budge,"In one of the following years: 1490, 1491, 149...",<NA>,Male,Human,Dark,<NA>,...,<NA>,<NA>,Ravenclaw,<NA>,"['Fungus-face' (by his mother), The Hermit of ...","[Mrs Budge (mother), Great-grandmother]","[Author, Potioneer]",None,None,None
4064,8080ef61-4c6c-46c1-9cf0-629d16a23110,character,zygmunt-budge-s-great-grandmother,Zygmunt Budge's great-grandmother,<NA>,<NA>,Female,Human,Grey,<NA>,...,<NA>,<NA>,Ravenclaw,<NA>,None,[Mrs Budge (granddaughter or granddaughter-in-...,None,None,None,None


In [16]:
import numpy as np

dict_aux = {'Gryffindor': ['Gryffindor', 'Gryffindor (likely)', 
'Gryffindor (possibly)', 'Gryffindor or Slytherin', 
'Gryffindor, Hufflepuff or Ravenclaw', 'Gryffindor, Hufflepuff,  or Slytherin', 
'Gryffindor, Hufflepuff, or Ravenclaw', 'Gryffindor, Hufflepuff, or Slytherin', 'Pub landlady'], 'Did not attend Hogwarts': [None], 
'Hogwarts School of Witchcraft and Wizardry': [None], 'Horned Serpent': ['Horned Serpent'], 'Hufflepuff': ['Hufflepuff', 'Hufflepuff (likely)', 
'Hufflepuff (possibly)', 'Hufflepuff or Ravenclaw', 'Hufflepuff or Slytherin', 'Hufflepuff, Ravenclaw or Slytherin'], 
'Pukwudgie': ['Pukwudgie'], 'Ravenclaw': ['Ravenclaw', 'Ravenclaw (likely)', 'Ravenclaw (possibly)', 
'Ravenclaw or Hufflepuff', 'Ravenclaw or Slytherin', 'Ravenclaw, Slytherin or Hufflepuff'], 'Slytherin': ['Slytherin', 
'Slytherin (likely)', 'Slytherin (most likely)', 'Slytherin (possibly)'], 'Thunderbird': ['Thunderbird'], 'Wampus': ['Wampus'], 'Unknown': [None]}

def mantenerCasas(cell):
    for elem in dict_aux: 
        if cell is not None or cell is not np.nan: 
            # print("Entre a este caso")
            if cell == elem: 
                return elem
            if cell in dict_aux[elem]:
                return elem

        else: 
            return None


In [17]:
aux = char['house'].apply(mantenerCasas)
aux

0       Ravenclaw
1       Ravenclaw
2       Ravenclaw
3       Ravenclaw
4       Ravenclaw
          ...    
4061    Ravenclaw
4062    Slytherin
4063    Ravenclaw
4064    Ravenclaw
4065    Ravenclaw
Name: house, Length: 4066, dtype: object

In [18]:
aux.unique()

array(['Ravenclaw', 'Gryffindor', 'Slytherin', 'Hufflepuff', 'Unknown',
       'Thunderbird', 'Did not attend Hogwarts', 'Horned Serpent',
       'Hogwarts School of Witchcraft and Wizardry', 'Pukwudgie',
       'Wampus'], dtype=object)

In [72]:
# Vamos a cambiar los nulls
char.fillna("missing", inplace=True)

# suma = char.isna().sum()
# print(suma)

In [73]:
char = char.apply(lambda x: x.astype(str).str.lower())
char

,id,type,slug,name,born,died,gender,species,hair_color,eye_color,...,animagus,boggart,house,patronus,alias_names,family_members,jobs,romances,titles,wands
0,a3701158-39ed-4687-8b9a-a799cfca5811,character,1992-gryffindor-vs-slytherin-quidditch-match-s...,1992 gryffindor vs slytherin quidditch match s...,missing,missing,missing,missing,missing,missing,...,missing,missing,hufflepuff,missing,missing,missing,missing,missing,missing,missing
1,e06d8f25-fae5-4970-884e-06ae2f53d707,character,1996-gryffindor-quidditch-keeper-trials-specta...,1996 gryffindor quidditch keeper trials specta...,missing,missing,missing,missing,missing,missing,...,missing,missing,hufflepuff,missing,missing,missing,missing,missing,missing,missing
2,5e97c6a0-e000-4fb9-8d66-37cd6942d103,character,unidentified-19th-century-flying-instructor,19th-century flying instructor,before 1890,missing,female,human,black,missing,...,missing,missing,hufflepuff,missing,missing,missing,['flying instructor at hogwarts school of witc...,missing,missing,missing
3,fb7aad1b-50d7-4362-a315-d7432ae35372,character,unidentified-19th-century-herbology-professor,19th-century herbology professor,missing,missing,female,human,red,missing,...,missing,missing,hufflepuff,missing,missing,missing,['professor of herbology at hogwarts school of...,missing,missing,"['unknown length, wood and core']"
4,41812dca-8341-4369-b286-5e27f35274b4,character,unidentified-19th-century-potions-professor,19th-century potions professor,missing,missing,male,human,brown,missing,...,missing,missing,hufflepuff,missing,missing,missing,['potions master at hogwarts school of witchcr...,missing,missing,missing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4061,936e7daf-ab6c-4d09-903e-0ba8091c336a,character,zoo-director,zoo director,missing,missing,male,human,missing,missing,...,missing,missing,hufflepuff,missing,missing,missing,['zoo director'],missing,['director'],missing
4062,65853c0b-6990-4908-93bb-a4f3139ea181,character,zubeida-khan,zubeida khan,"1 september 1975 - 31 august 1982,",missing,female,human,missing,missing,...,missing,missing,slytherin,missing,missing,missing,missing,missing,missing,missing
4063,41e4f443-adf6-4935-a9f5-481e2657cf30,character,zygmunt-budge,zygmunt budge,"in one of the following years: 1490, 1491, 149...",missing,male,human,dark,missing,...,missing,missing,hufflepuff,missing,"[""'fungus-face' (by his mother)"", 'the hermit ...","['mrs budge (mother)', 'great-grandmother']","['author', 'potioneer']",missing,missing,missing
4064,8080ef61-4c6c-46c1-9cf0-629d16a23110,character,zygmunt-budge-s-great-grandmother,zygmunt budge's great-grandmother,missing,missing,female,human,grey,missing,...,missing,missing,hufflepuff,missing,missing,['mrs budge (granddaughter or granddaughter-in...,missing,missing,missing,missing


In [74]:
# †
def quitar(cell):
    cell = cell.astype(str)
    resp = cell.replace(to_replace=r'[\!\#\$\%\&\(\)\'\"\*\+\-\.\/\:\;\<\=\>\?\@\^\`\{\|\}\~\†]', value=' ', regex=True)
    return resp

char = char.apply(quitar)
char

,id,type,slug,name,born,died,gender,species,hair_color,eye_color,...,animagus,boggart,house,patronus,alias_names,family_members,jobs,romances,titles,wands
0,a3701158 39ed 4687 8b9a a799cfca5811,character,1992 gryffindor vs slytherin quidditch match s...,1992 gryffindor vs slytherin quidditch match s...,missing,missing,missing,missing,missing,missing,...,missing,missing,hufflepuff,missing,missing,missing,missing,missing,missing,missing
1,e06d8f25 fae5 4970 884e 06ae2f53d707,character,1996 gryffindor quidditch keeper trials specta...,1996 gryffindor quidditch keeper trials specta...,missing,missing,missing,missing,missing,missing,...,missing,missing,hufflepuff,missing,missing,missing,missing,missing,missing,missing
2,5e97c6a0 e000 4fb9 8d66 37cd6942d103,character,unidentified 19th century flying instructor,19th century flying instructor,before 1890,missing,female,human,black,missing,...,missing,missing,hufflepuff,missing,missing,missing,[ flying instructor at hogwarts school of witc...,missing,missing,missing
3,fb7aad1b 50d7 4362 a315 d7432ae35372,character,unidentified 19th century herbology professor,19th century herbology professor,missing,missing,female,human,red,missing,...,missing,missing,hufflepuff,missing,missing,missing,[ professor of herbology at hogwarts school of...,missing,missing,"[ unknown length, wood and core ]"
4,41812dca 8341 4369 b286 5e27f35274b4,character,unidentified 19th century potions professor,19th century potions professor,missing,missing,male,human,brown,missing,...,missing,missing,hufflepuff,missing,missing,missing,[ potions master at hogwarts school of witchcr...,missing,missing,missing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4061,936e7daf ab6c 4d09 903e 0ba8091c336a,character,zoo director,zoo director,missing,missing,male,human,missing,missing,...,missing,missing,hufflepuff,missing,missing,missing,[ zoo director ],missing,[ director ],missing
4062,65853c0b 6990 4908 93bb a4f3139ea181,character,zubeida khan,zubeida khan,"1 september 1975 31 august 1982,",missing,female,human,missing,missing,...,missing,missing,slytherin,missing,missing,missing,missing,missing,missing,missing
4063,41e4f443 adf6 4935 a9f5 481e2657cf30,character,zygmunt budge,zygmunt budge,"in one of the following years 1490, 1491, 149...",missing,male,human,dark,missing,...,missing,missing,hufflepuff,missing,"[ fungus face by his mother , the hermit ...","[ mrs budge mother , great grandmother ]","[ author , potioneer ]",missing,missing,missing
4064,8080ef61 4c6c 46c1 9cf0 629d16a23110,character,zygmunt budge s great grandmother,zygmunt budge s great grandmother,missing,missing,female,human,grey,missing,...,missing,missing,hufflepuff,missing,missing,[ mrs budge granddaughter or granddaughter in...,missing,missing,missing,missing


Pasarlo a csv

In [51]:
# char.to_csv('harrypotter', sep='\t', encoding='utf-8')